In [3]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [2]:
df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_180630.txt')

In [4]:
cols={x:x.strip() for x in df.columns}
df_ren=df.rename(columns=cols)
df_small=df_ren[:1000]

Next we want to extract a time stampe from a given row in the dataframe.  The following is an example of how to do this for a particular row.  In the next frame we do this for each row.

In [246]:
pd.to_datetime(df_small.iloc[1][6]+' '+df_small.iloc[1][7],infer_datetime_format=True)

Timestamp('2018-06-23 04:00:00')

In [18]:
def get_datetime(x):
    return pd.to_datetime(x[6]+' '+x[7],infer_datetime_format=True)
df_small['datetime']=df_small.apply(get_datetime,axis=1)

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
df_small_clean=df_small.drop_duplicates(subset=['C/A','UNIT','SCP','STATION','LINENAME','datetime'])

At this point we should group by station and datetime **then** resample to make sure that we only have proper intervales of 4.

In [73]:
df_small_clean2=df_small_clean.groupby(['STATION','datetime'])[['EXITS']].sum()
df_small_clean3=df_small_clean2.reset_index()

Now we need to get diffs for each station, which means we need to find a list of stations, go by each one, sort by datetime, resample do a diff operation, etc.  The following takes in a dataframe like df_small_clean3 and a station and then gives back a dataframe for that particular station with the indices sorted by datetime.

In [66]:
def station_activity(df,station):
    df_station=df[df['STATION']==station]
    df_sort=df_station.sort_values(by=['datetime'])
    return df_sort

In [170]:
station_activity(df_small_clean3,'59 ST').head()

,STATION,datetime,EXITS
43,59 ST,2018-06-23 00:00:00,28605423
44,59 ST,2018-06-23 04:00:00,28605596
45,59 ST,2018-06-23 08:00:00,28605997
46,59 ST,2018-06-23 12:00:00,28607167
47,59 ST,2018-06-23 16:00:00,28608285


OK, now for each station, apply a re-sample and then a diff map and drop the first element.  First, make a dataframe for each station, combine these in a list.

In writing the following definitions we are requring the seconds to be zero.  This is just a simple way to get rid of the wierd entries.

In [172]:
def good_time_mask(df):
    masks=[]
    for i in range(len(df)):
        test=(df['datetime'].iloc[i].hour%4==0)&(df['datetime'].iloc[i].second==0)        
        masks.append(test)
    return pd.Series(masks)

def bad_time_mask(df):
    masks=[]
    for i in range(len(df)):
        test=(df['datetime'].iloc[i].hour%4!=0)&(df['datetime'].iloc[i].second==0)
        masks.append(test)
    return pd.Series(masks)

We Hereby Declare:  All bad-time index sets will just be mapped to good time indices by hand.

In [168]:
m3=good_time_mask(df_small_clean3)
df_small_clean3[m3];

Below we demonstrate how the time deltas can be used to shift the timestamp.

In [250]:
ts=df_small_clean3.iloc[0][1]
t_delta=datetime.timedelta(0,0,0,0,0,2)
ts.second, ts-t_delta

(0, Timestamp('2018-06-23 00:00:00'))

In [ ]:
In the next step we write a function to shift the times on the rows of the bad times.

In [247]:
def shift_time(row):
    t_delta=datetime.timedelta(0,0,0,0,0,2)
    ts=row[1]
    row[1]=ts-t_delta
    return row

Let's check to see that shift_time works on a particular row.

In [147]:
shift_time(df_small_clean3.iloc[0])

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


STATION              5 AV/59 ST
datetime    2018-06-23 00:00:00
EXITS                  60941315
Name: 0, dtype: object

Finally, we write a function which takes in a dataframe and sets all the times to the proper intervales.  Basically, we break the dataframe into 'bad' and 'good' pieces and apply the shift to the bad pieces.

In [251]:
def bad_to_good(df):
    gm=good_time_mask(df)
    bm=bad_time_mask(df)
    g_df=df[gm]
    b_df=df[bm]
    b2g_df=b_df.apply(shift_time,axis=1)
    fin=pd.concat([g_df,b2g_df],ignore_index=True)
    return fin

In [214]:
df_small_clean4=bad_to_good(df_small_clean3)

Next, go by station and for each station, apply the diff operation, obtaining a list of dataframes containing the diff values.

In [309]:
station_diffs_dct={}
for st in stations:
    sa=station_activity(df_small_clean4,st)
    st_diffs=pd.DataFrame(sa['EXITS'].diff())
    sa['EXITS']=st_diffs['EXITS']
    sa=sa.rename(columns={'EXITS':'diffs'})
    station_diffs_dct[st]=sa

Need to add a column for weekday and an hour, dropna, group by weekday and hour.

In [311]:
st_df=station_diffs_dct['59 ST'].dropna()
st_df_filt=st_df[np.abs(st_df.diffs)<20000]
st_df_filt['weekday']=st_df_filt['datetime'].apply(lambda x: x.weekday())
st_df_filt['hour']=st_df_filt['datetime'].apply(lambda x: x.hour)
st_df_clean2=st_df_filt.groupby(['weekday','hour'])['diffs'].mean()

In [319]:
dct_clean={}
for st in stations:
    st_df=station_diffs_dct[st].dropna()
    st_df_filt=st_df[np.abs(st_df.diffs)<20000]
    st_df_filt['weekday']=st_df_filt['datetime'].apply(lambda x: x.weekday())
    st_df_filt['hour']=st_df_filt['datetime'].apply(lambda x: x.hour)
    st_df_clean2=st_df_filt.groupby(['weekday','hour'])['diffs'].mean()
    dct_clean[st]=st_df_clean2

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [359]:
def activity_by_time(day,hour):
    #We must convert a 24hour of the day to one of 0,4,8...
    h2=((hour//4+1)*4)%24
    exits=[]
    for st in stations:
        leaving=dct_clean[st][day][h2]
        exits.append([st,leaving])
    sort_exits=sorted(exits,key=lambda x: x[1])[::-1]    
    return sort_exits

When inputting the time remember that Monday is zero, Tuesday is 1, etc.  The hour may be input as an integer and it internally converts to one of the time intervals [0,4], [4,8], etc.  The result appears in order from largest flux to smallest.

In [361]:
activity_by_time(3,23)

[['59 ST', 637.0], ['5 AV/59 ST', 177.0]]